# Ejercicio aplicado de DataFrames y Spark SQL

In [28]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Df')\
        .getOrCreate()

In [10]:
## Importa el csv de "data/WorldCupPlayers.csv"
## Visualiza los datos
filepath = '../datasets/'
df = spark.read.csv(filepath + 'WorldCupPlayers.csv', sep=",", header=True, inferSchema=True)
df.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 

In [11]:
## ¿que tipo de datos contiene cada variable?
df.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [14]:
## ¿Cuantos registros hay?
df.count()

37784

In [25]:
## Obtén los principales estadísticos de Position
#df.filter(df["Position"] != 'NULL').show()
df_mode = df.groupBy('position').count()
df_mode.show()

+--------+-----+
|position|count|
+--------+-----+
|    NULL|33641|
|      GK| 2441|
|       C| 1510|
|     GKC|  192|
+--------+-----+



In [26]:
## Slecciona y muestra los registros distintos de 'Player Name','Coach Name'
# Seleccionar registros distintos para 'Player Name' y 'Coach Name'
distinct_players_coaches = df.select('Player Name', 'Coach Name').dropDuplicates()

# Mostrar los resultados
distinct_players_coaches.show()

+--------------------+--------------------+
|         Player Name|          Coach Name|
+--------------------+--------------------+
|    Arturo FERNANDEZ| BRU Francisco (ESP)|
|Cayetano CARRERAS...|DURAND LAGUNA Jos...|
|  Ernesto MASCHERONI|SUPPICI Alberto (...|
|          Aziz FAHMY|   McREA James (SCO)|
|        Gyula POLGAR|    NADAS Odon (HUN)|
|  Ernesto ALBARRACIN|PASCUCCI Felipe (...|
| Armando CASTELLAZZI|POZZO Vittorio (ITA)|
|     Jaroslav BOUCEK|   PETRU Karel (TCH)|
|           Erwin NYC|  KALUZA Jozef (POL)|
|     Stanislaw BARAN|  KALUZA Jozef (POL)|
|     Fernando ROLDAN|BUCCIARDI Arturo ...|
|            Joe MACA|  JEFFREY Bill (SCO)|
|               INDIO|  MOREIRA Zeze (BRA)|
|      Rene DEREUDDRE|PIBAROT Pierre (FRA)|
|    Anton MALATINSKY|    CEJP Josef (TCH)|
|    Alberto MARIOTTI|LORENZO Juan Carl...|
|  Alfredo DI STEFANO|HERRERA Helenio (...|
|             FIDELIS| FEOLA Vicente (BRA)|
|     Stoyan YORDANOV|BOZHKOV Stefan (BUL)|
|      Wim RIJSBERGEN| MICHELS R

In [33]:
## ¿Cuantos partidos con el ID de 1096 ha habido?
# Filtrar por MatchID 1096 y contar los registros
count = df.filter(df.MatchID == 1096).count()
count

33

In [34]:
## Muestra los datos donde la posicion haya sido C y el evento sea G40
filter_position = df.filter((df.Position == 'C') & (df.Event.contains('G40')))
filter_position.show()

+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|RoundID|MatchID|Team Initials|          Coach Name|Line-up|Shirt Number|     Player Name|Position|Event|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|    201|   1089|          PAR|DURAND LAGUNA Jos...|      S|           0|Luis VARGAS PENA|       C| G40'|
|    429|   1175|          HUN|  DIETZ Karoly (HUN)|      S|           0|   Gyorgy SAROSI|       C| G40'|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+



In [35]:
## Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20
filter_match = df.filter((df.MatchID >= 20))
filter_match.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     NULL|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    NULL|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    NULL|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    NULL|     NULL|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     NULL|
|    201| 